In [70]:
#pip install ddddocr -i https://pypi.tuna.tsinghua.edu.cn/simple
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar, DateEntry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time, ddddocr

def registration():
    hops = hops_dp.get()
    getdate =  cal.get_date()
    
    date_LB = tk.Label(root, text = getdate, font='Arial 15', width='25')
    date_LB.grid(column=1, row=2)
    
    RID = 'RXXXXXXXXX' #這邊輸入身分證字號
    Birth = 'XXXX'#這邊輸入生日
    if hops == "台南市立醫院":
        global driver
        try:
            url = 'https://www.tmh.org.tw/TMH2016/RegDr.aspx?Kind=2&dept=MH&Date=' + getdate + '&Sect=2113&Noon=1'
            driver = webdriver.Firefox(executable_path="geckodriver")
            driver.get(url)
            
            input1_RID = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_TB_ID')
            input1_RID.send_keys(RID)
        
            code = driver.find_element(By.ID,'ctl00_ContentPlaceHolder1_TB_CODE')
            imgcode = driver.find_element(By.ID,'ctl00_ContentPlaceHolder1_ImgChkCode')
            imgcode.screenshot("code.png")
            time.sleep(1)
        
            ocr = ddddocr.DdddOcr()
            with open("code.png", "rb") as fp:
                image = fp.read()
            catch = ocr.classification(image)
            code.send_keys(catch)
            time.sleep(10)
        
            btn1 = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_LB_Next')
            btn1.click()
            time.sleep(1)
            
            input_Birth = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_TB_Birthday')
            input_Birth.send_keys(Birth)
            time.sleep(1)
            
            btn2 = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_LB_Reg')
            btn2.click()
            time.sleep(1)
            
            top = tk.Toplevel()
            LB_OK = tk.Label(top, text='掛號成功!' , font='Arial 15', width=15).pack()
            top.mainloop()
            
        except:
            driver.close()
            date_LB.config(text = "此日期無門診，請重新選擇!")
            
    elif hops == "新營醫院":
        try:
            url = 'https://newreg.syh.mohw.gov.tw/oinetreg/OINetReg.Reg/Reg_RegTable.aspx?HID=F&Date=' + getdate + '&Noon=0&DivDr=0203&Way=Date'
            driver = webdriver.Firefox(executable_path="geckodriver")
            driver.get(url)

            driver.switch_to.frame(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_iframeRegTable'))
            tr = driver.find_element(By.CLASS_NAME, 'trRegSchDay')
            a2 = tr.find_element(By.TAG_NAME, 'a')
            a2.click()

            handles = driver.window_handles
            driver.switch_to.window(handles[-1])

            input1_RID = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtIDNOorPatientID')
            input1_RID.send_keys(RID)

            select1 = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_dd1BirthM')
            options1 = select1.find_elements(By.TAG_NAME, 'option')
            options1[8].click()

            select2 = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_dd1BirthD')
            options2 = select2.find_elements(By.TAG_NAME, 'option')
            options2[17].click()
            
            code = driver.find_element(By.ID,'ctl00_ContentPlaceHolder1_txtVerificationCode')
            imgcode = driver.find_element(By.ID,'ctl00_ContentPlaceHolder1_imgVerificationCode')
            imgcode.screenshot("code.png")
            time.sleep(1)
        
            ocr = ddddocr.DdddOcr()
            with open("code.png", "rb") as fp:
                image = fp.read()
            catch = ocr.classification(image)
            code.send_keys(catch.upper())
            time.sleep(10)
            
            btn1 = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnReg')
            btn1.click()
            time.sleep(10)
            
            top = tk.Toplevel()
            LB_OK = tk.Label(top, text='掛號成功!' , font='Arial 15', width=15).pack()
            top.mainloop()
        except:
            driver.close()
            date_LB.config(text = "此日期無門診，請重新選擇!")
    else:
        pass
    
#===主視窗
root = tk.Tk()
root.title('自動回診掛號系統')
root.geometry('500x350')

LB1 = tk.Label(root, text='歡迎使用 ' , font='Arial 15', width=15)
LB1.grid(column=0, row=0)

hopspital_LB = tk.Label(root, text='醫院 : ' , font='Arial 16', width=15)
hopspital_LB.grid(column=0, row=1)

nums = ['台南市立醫院', '新營醫院'] 
sel_nums = tk.StringVar()
hops_dp = ttk.Combobox(root, textvariable='', values=nums, font='Arial 16', width='15')
hops_dp.current(0)
hops_dp.grid(column=1, row=1)

date_LB = tk.Label(root, text='看診日期 : ' , font='Arial 16', width=15)
date_LB.grid(column=0, row=2)

cal = Calendar(root, selectmode = 'day',date_pattern = 'y/mm/dd')
cal.grid(column=1, row=3)
   
#tk.Button(root,text="選擇日期",command=grad_date).grid(column=2, row=3)
tk.Button(root,text="前往網頁掛號", font='Arial 18', width='15',command=registration).grid(column=1, row=5)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Wantzu\AppData\Local\Temp\ipykernel_16432\355290319.py", line 74, in registration
    driver.switch_to.window(handles[-1])
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Wantzu\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Wantzu\AppData\Local\Temp\ipykernel_16432\355290319.py", line 107, in registration
    driver.close()
  File "C:\Users\Wantzu\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 551, in close
    self.execute(Command.CLOSE)
  File "C:\Users\Wantzu\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "C:\Users\Wantzu\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 245, in check_res